In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NLP SPAM').getOrCreate()

In [4]:
data = spark.read.csv('SMSSpamCollection',sep='\t',inferSchema=True)

In [7]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [11]:
data = data.withColumnRenamed('_c0','class')

data = data.withColumnRenamed('_c1','text')

In [12]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [13]:
# need to clean data

In [14]:
from pyspark.sql.functions import *

In [16]:
data = data.withColumn('length',length('text'))

In [17]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [18]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [19]:
from pyspark.ml.feature import Tokenizer, IDF, StopWordsRemover, CountVectorizer,StringIndexer

In [62]:
tokenizer = Tokenizer(inputCol='text',outputCol='text_tokens')
stopwords = StopWordsRemover(inputCol='text_tokens',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='count_token')
idf = IDF(inputCol= 'count_token', outputCol='count_vec')

ham_sam_num = StringIndexer(inputCol='class',outputCol='label')

In [63]:
 from pyspark.ml.feature import VectorAssembler

In [64]:
clean_up = VectorAssembler(inputCols=['count_token','length'],outputCol='features')

In [65]:
from pyspark.ml.classification import NaiveBayes

In [66]:
nb =NaiveBayes()

In [67]:
from pyspark.ml.pipeline import Pipeline

In [68]:
pipeline_data = Pipeline (stages=[ham_sam_num,tokenizer, stopwords,count_vec,idf,clean_up] )

In [69]:
cleaner = pipeline_data.fit(data)

In [70]:
clean_data = cleaner.transform(data)

In [71]:
clean_data = clean_data.select('label','features')

In [72]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [80]:
training, testing = clean_data.randomSplit([0.7,0.3])

In [81]:
spam_detector  = nb.fit(training)

In [83]:
test_result = spam_detector.transform(testing)

In [84]:
test_result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-194.33969827423...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-191.59953233804...|[0.99999167696330...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-179.71858836942...|[0.99999999181778...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-176.49240176027...|[0.99999999723710...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-214.92318282230...|[0.99999998189206...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-249.80457222075...|[0.99999999999998...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-63.497324176425...|[0.99999932941014...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-258.73464078410...|[1.0,1.5867894193...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-265.38518957343...|[0.99999999978425...|       0.0|
|  0.0|(13424,[0

In [88]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [90]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_result)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9716530541945847


In [ ]:
# Hence it is Fine 